# Using IBM watsonx.governance metrics toolkit to evaluate the quality of your Large Language Models

In [ ]:
!pip install --upgrade ibm-watson-machine-learning   | tail -n 1
!pip install --upgrade ibm-watson-openscale --no-cache | tail -n 1
!pip install --upgrade ibm-metrics-plugin --no-cache | tail -n 1

In [ ]:
!pip install --upgrade evaluate --no-cache | tail -n 1
!pip install --upgrade rouge_score --no-cache | tail -n 1
!pip install --upgrade textstat --no-cache | tail -n 1
!pip install --upgrade sacrebleu --no-cache | tail -n 1
!pip install --upgrade sacremoses --no-cache | tail -n 1
!pip install --upgrade datasets==2.10.0 --no-cache | tail -n 1

In [ ]:
!pip install boto3 -U --no-cache | tail -n 1

In [ ]:
import warnings
warnings.filterwarnings('ignore')

## Provision services and configure credentials

If you have not already, provision an instance of IBM Watson OpenScale using the [OpenScale link in the Cloud catalog](https://cloud.ibm.com/catalog/services/watson-openscale).

Your Cloud API key can be generated by going to the [**Users** section of the Cloud console](https://cloud.ibm.com/iam#/users). From that page, click your name, scroll down to the **API Keys** section, and click **Create an IBM Cloud API key**. Give your key a name and click **Create**, then copy the created key and paste it below.

**NOTE:** You can also get OpenScale `API_KEY` using IBM CLOUD CLI.

How to install IBM Cloud (bluemix) console: [instruction](https://console.bluemix.net/docs/cli/reference/ibmcloud/download_cli.html#install_use)

How to get api key using console:
```
bx login --sso
bx iam api-key-create 'my_key'
```

In [ ]:
CLOUD_API_KEY = "xxxxx"
IAM_URL="https://iam.ng.bluemix.net/oidc/token"

## IBM watsonx.governance authentication

In [ ]:
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator,BearerTokenAuthenticator

from ibm_watson_openscale import *
from ibm_watson_openscale.supporting_classes.enums import *
from ibm_watson_openscale.supporting_classes import *


authenticator = IAMAuthenticator(apikey=CLOUD_API_KEY)
client = APIClient(authenticator=authenticator)
client.version

# Common Imports

In [ ]:
from ibm_metrics_plugin.metrics.llm.utils.constants import LLMTextMetricGroup
from ibm_metrics_plugin.metrics.llm.utils.constants import  LLMGenerationMetrics
from ibm_metrics_plugin.metrics.llm.utils.constants import LLMSummarizationMetrics
from ibm_metrics_plugin.metrics.llm.utils.constants import LLMQAMetrics
from ibm_metrics_plugin.metrics.llm.utils.constants import LLMClassificationMetrics
from ibm_metrics_plugin.metrics.llm.utils.constants import HAP_SCORE
from ibm_metrics_plugin.metrics.llm.utils.constants import PII_DETECTION

# Evaluating Summarization output from AWS/anthropic.claude-v2

## Test data containing the summarization output from model and the reference data

In [ ]:
!rm -fr llm_content.csv
!wget "https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/watsonx/llm_content.csv"

In [ ]:
import pandas as pd
import numpy as np
llm_data_all = pd.read_csv("llm_content.csv")
llm_data_all.head()

In [ ]:
llm_data = llm_data_all.head(10)
llm_data.head()

In [ ]:
import boto3, json

# AWS Access credentials

In [ ]:
aws_access_key_id = 'xxxxxx'
aws_secret_access_key = 'xxxxxx'

In [ ]:
session = boto3.Session()

## Create the bedrock service client

In [ ]:
bedrock = session.client(service_name='bedrock', 
                         aws_access_key_id = aws_access_key_id, 
                         aws_secret_access_key = aws_secret_access_key, 
                         region_name = 'us-east-1',
                         endpoint_url = 'https://bedrock.us-east-1.amazonaws.com')

## List the available foundation models to use

In [ ]:
fm_model_list = bedrock.list_foundation_models()

In [ ]:
fm_model_names = [x['modelId'] for x in fm_model_list['modelSummaries']]
print(*fm_model_names, sep = "\n")

## Let's use `anthropic.claude-v2` model for inferencing

In [ ]:
modelId = 'anthropic.claude-v2' # change this to use a different version from the model provider
accept = 'application/json'
contentType = 'application/json'

## Create bedrock-runtime client

In [ ]:
bedrock_runtime = session.client(service_name='bedrock-runtime', 
                         aws_access_key_id = aws_access_key_id, 
                         aws_secret_access_key = aws_secret_access_key, 
                         region_name = 'us-east-1',
                         endpoint_url = 'https://bedrock-runtime.us-east-1.amazonaws.com')

## Prompt for inferencing

In [ ]:
def get_prompt(text):
    prompt = f"""Human: Please provide a summary of the following text with maximum of 20 words.
    
{text}
    
Assistant:"""
    return prompt

## Evaluate the prompt

In [ ]:
def prompt_evaluation(text):
    prompt = get_prompt(text)
    body = json.dumps({"prompt": prompt,
                     "max_tokens_to_sample":2048,
                     "temperature":0.1,
                     "top_k":250,
                     "top_p":0.5,
                     "stop_sequences":[]
                      }) 
    response = bedrock_runtime.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
    response_body = json.loads(response.get('body').read())
    completion = response_body['completion']
    summary = completion
    if '\n\n' in completion:
        summary = completion.split("\n\n")[1]
    print('-----')    
    print(summary)
    print('-----')
    return summary

## Append the generated summary, obtained as part of the inferencing against `anthropic.claude-v2` model, to the LLM data set

In [ ]:
llm_data['anthropic_generated_summary'] = llm_data['input_text'].apply(prompt_evaluation)

In [ ]:
llm_data.head()

## Get the necessary data for evaluating the quality of the prompt template

In [ ]:
df_input = llm_data[['input_text']].copy()
df_output = llm_data[['anthropic_generated_summary']].copy()
df_reference = llm_data[['reference_summary_2']].copy()

## Metrics configuration for evaluation

In [ ]:
metric_config = {   
    "configuration": {
        LLMTextMetricGroup.SUMMARIZATION.value: {
            LLMSummarizationMetrics.ROUGE_SCORE.value: {},
            LLMSummarizationMetrics.SARI.value: {},
            LLMSummarizationMetrics.METEOR.value: {},
            LLMSummarizationMetrics.NORMALIZED_RECALL.value: {},
            LLMSummarizationMetrics.NORMALIZED_PRECISION.value: {},
            LLMSummarizationMetrics.NORMALIZED_F1_SCORE.value: {},
            LLMSummarizationMetrics.COSINE_SIMILARITY.value: {},
            LLMSummarizationMetrics.JACCARD_SIMILARITY.value: {},
            LLMSummarizationMetrics.BLEU.value: {},
            LLMSummarizationMetrics.FLESCH.value: {}
        }
    }
}

## Summarization Metrics Evaluation

In [ ]:
import json
result = client.llm_metrics.compute_metrics(metric_config,sources = df_input, predictions = df_output, references = df_reference)

## Evaluated Metrics

In [ ]:
print(json.dumps(result,indent=2))

# Evaluating Content Generation output from the Foundation Model

## Test data containing the content generation output from model and the reference data

In [ ]:
!rm -fr llm_content_generation.csv
!wget "https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/watsonx/llm_content_generation.csv"

In [ ]:
data = pd.read_csv("llm_content_generation.csv")
data.head()

In [ ]:
df_input = data[['question']].copy()
df_output = data[['generated_text']].copy()
df_reference = data[['reference_text']].copy()

## Metrics configuration for evaluation

In [ ]:
metric_config = {   
    #All Common parameters goes here 
    "configuration": {        
        LLMTextMetricGroup.GENERATION.value: { # metric group   
            LLMGenerationMetrics.BLEU.value: {},
            LLMGenerationMetrics.ROUGE_SCORE.value: {},
            LLMGenerationMetrics.FLESCH.value: {},
            LLMGenerationMetrics.METEOR.value: {},            
            LLMGenerationMetrics.NORMALIZED_RECALL.value: {},
            LLMGenerationMetrics.NORMALIZED_PRECISION.value: {},
            LLMGenerationMetrics.NORMALIZED_F1_SCORE.value: {}            
        }    
    }
}

## Content Generation Metrics Evaluation

In [ ]:
result = client.llm_metrics.compute_metrics(metric_config,df_input,df_output, df_reference)

## Evaluated Metrics

In [ ]:
print(json.dumps(result,indent=2))

# Evaluating Question and Answering output from the Foundation Model

## Test data containing the question and answer output from model and the reference data

In [ ]:
!rm -fr llm_content_qa.csv
!wget "https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/watsonx/llm_content_qa.csv"

In [ ]:
data = pd.read_csv("llm_content_qa.csv")
data.head()

In [ ]:
df_input = data[['question']].copy()
df_output = data[['answers']].copy()
df_reference = data[['answers']].copy()

## Metrics configuration for evaluation

In [ ]:
metric_config = {   
    #All Common parameters goes here 
    "configuration": {        
        LLMTextMetricGroup.QA.value: { # metric group   
            LLMQAMetrics.EXACT_MATCH.value: {},
            LLMQAMetrics.ROUGE_SCORE.value: {},
            LLMQAMetrics.BLEU.value: {}          
        }    
    }
}

## Question and Answering Metrics Evaluation

In [ ]:
result = client.llm_metrics.compute_metrics(metric_config,df_input,df_output, df_reference)

## Evaluated Metrics

In [ ]:
print(json.dumps(result,indent=2))

# Evaluating Text Classification output from the Foundation Model

## Test data containing the text classification output from model and the reference data

In [ ]:
!rm -fr llm_content_classification.csv
!wget "https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/watsonx/llm_content_classification.csv"

In [ ]:
data = pd.read_csv("llm_content_classification.csv")
data.head()

In [ ]:
data['label'] = data['label'].replace({'ham': 0, 'spam': 1})

In [ ]:
df_input = data[['text']].copy()
df_output = data[['label']].copy()
df_reference = data[['label']].copy()

## Make some realistic reference column

In [ ]:
shuffled_column = df_reference['label'].sample(frac=1).reset_index(drop=True)
df_reference['label'] = shuffled_column

## Metrics configuration for evaluation

In [ ]:
metric_config = {   
    #All Common parameters goes here 
    "configuration": {        
        LLMTextMetricGroup.CLASSIFICATION.value: { # metric group   
            LLMClassificationMetrics.ACCURACY.value: {},
            LLMClassificationMetrics.PRECISION.value: {},
            LLMClassificationMetrics.RECALL.value: {},
            LLMClassificationMetrics.F1_SCORE.value: {},
            LLMClassificationMetrics.MATTHEWS_CORRELATION.value: {},            
        }    
    }
}

## Text Classification Metrics Evaluation

In [ ]:
result = client.llm_metrics.compute_metrics(metric_config,df_input,df_output, df_reference)

## Evaluated Metrics

In [ ]:
print(json.dumps(result,indent=2))

Author: kishore.patel@in.ibm.com , ravi.chamarthy@in.ibm.com